<a href="https://colab.research.google.com/github/suminohh/OFDS/blob/main/OFDS_cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
#main reference: https://towardsdatascience.com/preprocessing-text-data-using-python-576206753c28

import pandas as pd
import numpy as np
import nltk
!pip install fasttext
import fasttext
import string
import re
!pip install contractions
import contractions

In [14]:
df = pd.read_csv('OFDSpy.csv')
print(df)

#https://datatofish.com/import-csv-file-python-using-pandas/
#select column and save as separate dataset
response = pd.DataFrame(df, columns= ['response'])
print(response)

for col in df.columns:
    print(col, df[col].isnull().sum())

     serv_name        serv_rating  ...             hh_income  gender
0      GrubHub            4 Stars  ...       $25,000-$49,999    Male
1     DoorDash            3 Stars  ...       $25,000-$49,999  Female
2     DoorDash  5 Stars (Highest)  ...       $25,000-$49,999    Male
3     DoorDash            4 Stars  ...       $25,000-$49,999  Female
4    Postmates            4 Stars  ...             $0-$9,999  Female
..         ...                ...  ...                   ...     ...
230   DoorDash            3 Stars  ...       $25,000-$49,999  Female
231   DoorDash  5 Stars (Highest)  ...       $10,000-$24,999  Female
232  Postmates  5 Stars (Highest)  ...       $75,000-$99,999    Male
233    GrubHub            3 Stars  ...       $25,000-$49,999  Female
234   DoorDash            4 Stars  ...  Prefer not to answer    Male

[235 rows x 15 columns]
                                              response
0                  Good Always very hot and very tasty
1    When drivers pickup food from th

In [80]:
ds = df.loc[:, ['response', 'serv_name', 'serv_rating', 'order_freq', 'pop_density', 'avg_order', 'avg_tip', 'serv_rating', 'exp_sentiment']]

#expand contraction and tokenizes contracted words together so we need to convert it into a string
ds['no_contract'] = ds['response'].apply(lambda x:
[contractions.fix(word) for word in x.split()])
ds.head()

,response,serv_name,serv_rating,order_freq,pop_density,avg_order,avg_tip,serv_rating,exp_sentiment,no_contract
0,Good Always very hot and very tasty,GrubHub,4 Stars,Less than 4 times a month,Suburban,$21 - $25,16% - 20%,4 Stars,Very positive,"[Good, Always, very, hot, and, very, tasty]"
1,When drivers pickup food from the restaurant b...,DoorDash,3 Stars,Less than 4 times a month,Suburban,$30+,16% - 20%,3 Stars,Neutral,"[When, drivers, pickup, food, from, the, resta..."
2,ice cream was cold and I really liked it,DoorDash,5 Stars (Highest),3 - 4 times a week,Suburban,$11 - $15,0% - 5%,5 Stars (Highest),Very positive,"[ice, cream, was, cold, and, I, really, liked,..."
3,I order food through a third party to receive ...,DoorDash,4 Stars,Less than 4 times a month,Suburban,$21 - $25,6% - 10%,4 Stars,Positive,"[I, order, food, through, a, third, party, to,..."
4,I chose Food Delivery which is a cheap and fas...,Postmates,4 Stars,5+ times a week,Suburban,$7 - $10,16% - 20%,4 Stars,Positive,"[I, chose, Food, Delivery, which, is, a, cheap..."


In [81]:
#convert tokenization back to string
ds['response_str'] = [' '.join(map (str, no_contract)) for no_contract in 
ds['no_contract']]
ds.head()

,response,serv_name,serv_rating,order_freq,pop_density,avg_order,avg_tip,serv_rating,exp_sentiment,no_contract,response_str
0,Good Always very hot and very tasty,GrubHub,4 Stars,Less than 4 times a month,Suburban,$21 - $25,16% - 20%,4 Stars,Very positive,"[Good, Always, very, hot, and, very, tasty]",Good Always very hot and very tasty
1,When drivers pickup food from the restaurant b...,DoorDash,3 Stars,Less than 4 times a month,Suburban,$30+,16% - 20%,3 Stars,Neutral,"[When, drivers, pickup, food, from, the, resta...",When drivers pickup food from the restaurant b...
2,ice cream was cold and I really liked it,DoorDash,5 Stars (Highest),3 - 4 times a week,Suburban,$11 - $15,0% - 5%,5 Stars (Highest),Very positive,"[ice, cream, was, cold, and, I, really, liked,...",ice cream was cold and I really liked it
3,I order food through a third party to receive ...,DoorDash,4 Stars,Less than 4 times a month,Suburban,$21 - $25,6% - 10%,4 Stars,Positive,"[I, order, food, through, a, third, party, to,...",I order food through a third party to receive ...
4,I chose Food Delivery which is a cheap and fas...,Postmates,4 Stars,5+ times a week,Suburban,$7 - $10,16% - 20%,4 Stars,Positive,"[I, chose, Food, Delivery, which, is, a, cheap...",I chose Food Delivery which is a cheap and fas...


In [82]:
#tokenize string 
from nltk.tokenize import word_tokenize
nltk.download('punkt')
ds['tokenized_response'] = ds['response_str'].apply(word_tokenize)
ds.head()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


,response,serv_name,serv_rating,order_freq,pop_density,avg_order,avg_tip,serv_rating,exp_sentiment,no_contract,response_str,tokenized_response
0,Good Always very hot and very tasty,GrubHub,4 Stars,Less than 4 times a month,Suburban,$21 - $25,16% - 20%,4 Stars,Very positive,"[Good, Always, very, hot, and, very, tasty]",Good Always very hot and very tasty,"[Good, Always, very, hot, and, very, tasty]"
1,When drivers pickup food from the restaurant b...,DoorDash,3 Stars,Less than 4 times a month,Suburban,$30+,16% - 20%,3 Stars,Neutral,"[When, drivers, pickup, food, from, the, resta...",When drivers pickup food from the restaurant b...,"[When, drivers, pickup, food, from, the, resta..."
2,ice cream was cold and I really liked it,DoorDash,5 Stars (Highest),3 - 4 times a week,Suburban,$11 - $15,0% - 5%,5 Stars (Highest),Very positive,"[ice, cream, was, cold, and, I, really, liked,...",ice cream was cold and I really liked it,"[ice, cream, was, cold, and, I, really, liked,..."
3,I order food through a third party to receive ...,DoorDash,4 Stars,Less than 4 times a month,Suburban,$21 - $25,6% - 10%,4 Stars,Positive,"[I, order, food, through, a, third, party, to,...",I order food through a third party to receive ...,"[I, order, food, through, a, third, party, to,..."
4,I chose Food Delivery which is a cheap and fas...,Postmates,4 Stars,5+ times a week,Suburban,$7 - $10,16% - 20%,4 Stars,Positive,"[I, chose, Food, Delivery, which, is, a, cheap...",I chose Food Delivery which is a cheap and fas...,"[I, chose, Food, Delivery, which, is, a, cheap..."


In [83]:
#convert characters to lowercase
ds['lower'] = ds['tokenized_response'].apply(lambda x: [word.lower() for word in x])
ds.head()

,response,serv_name,serv_rating,order_freq,pop_density,avg_order,avg_tip,serv_rating,exp_sentiment,no_contract,response_str,tokenized_response,lower
0,Good Always very hot and very tasty,GrubHub,4 Stars,Less than 4 times a month,Suburban,$21 - $25,16% - 20%,4 Stars,Very positive,"[Good, Always, very, hot, and, very, tasty]",Good Always very hot and very tasty,"[Good, Always, very, hot, and, very, tasty]","[good, always, very, hot, and, very, tasty]"
1,When drivers pickup food from the restaurant b...,DoorDash,3 Stars,Less than 4 times a month,Suburban,$30+,16% - 20%,3 Stars,Neutral,"[When, drivers, pickup, food, from, the, resta...",When drivers pickup food from the restaurant b...,"[When, drivers, pickup, food, from, the, resta...","[when, drivers, pickup, food, from, the, resta..."
2,ice cream was cold and I really liked it,DoorDash,5 Stars (Highest),3 - 4 times a week,Suburban,$11 - $15,0% - 5%,5 Stars (Highest),Very positive,"[ice, cream, was, cold, and, I, really, liked,...",ice cream was cold and I really liked it,"[ice, cream, was, cold, and, I, really, liked,...","[ice, cream, was, cold, and, i, really, liked,..."
3,I order food through a third party to receive ...,DoorDash,4 Stars,Less than 4 times a month,Suburban,$21 - $25,6% - 10%,4 Stars,Positive,"[I, order, food, through, a, third, party, to,...",I order food through a third party to receive ...,"[I, order, food, through, a, third, party, to,...","[i, order, food, through, a, third, party, to,..."
4,I chose Food Delivery which is a cheap and fas...,Postmates,4 Stars,5+ times a week,Suburban,$7 - $10,16% - 20%,4 Stars,Positive,"[I, chose, Food, Delivery, which, is, a, cheap...",I chose Food Delivery which is a cheap and fas...,"[I, chose, Food, Delivery, which, is, a, cheap...","[i, chose, food, delivery, which, is, a, cheap..."


In [84]:
#remove punctuation
punc = string.punctuation
ds['no_punc'] = ds['lower'].apply(lambda x: [word for word in x if word not in punc])
ds.head()

,response,serv_name,serv_rating,order_freq,pop_density,avg_order,avg_tip,serv_rating,exp_sentiment,no_contract,response_str,tokenized_response,lower,no_punc
0,Good Always very hot and very tasty,GrubHub,4 Stars,Less than 4 times a month,Suburban,$21 - $25,16% - 20%,4 Stars,Very positive,"[Good, Always, very, hot, and, very, tasty]",Good Always very hot and very tasty,"[Good, Always, very, hot, and, very, tasty]","[good, always, very, hot, and, very, tasty]","[good, always, very, hot, and, very, tasty]"
1,When drivers pickup food from the restaurant b...,DoorDash,3 Stars,Less than 4 times a month,Suburban,$30+,16% - 20%,3 Stars,Neutral,"[When, drivers, pickup, food, from, the, resta...",When drivers pickup food from the restaurant b...,"[When, drivers, pickup, food, from, the, resta...","[when, drivers, pickup, food, from, the, resta...","[when, drivers, pickup, food, from, the, resta..."
2,ice cream was cold and I really liked it,DoorDash,5 Stars (Highest),3 - 4 times a week,Suburban,$11 - $15,0% - 5%,5 Stars (Highest),Very positive,"[ice, cream, was, cold, and, I, really, liked,...",ice cream was cold and I really liked it,"[ice, cream, was, cold, and, I, really, liked,...","[ice, cream, was, cold, and, i, really, liked,...","[ice, cream, was, cold, and, i, really, liked,..."
3,I order food through a third party to receive ...,DoorDash,4 Stars,Less than 4 times a month,Suburban,$21 - $25,6% - 10%,4 Stars,Positive,"[I, order, food, through, a, third, party, to,...",I order food through a third party to receive ...,"[I, order, food, through, a, third, party, to,...","[i, order, food, through, a, third, party, to,...","[i, order, food, through, a, third, party, to,..."
4,I chose Food Delivery which is a cheap and fas...,Postmates,4 Stars,5+ times a week,Suburban,$7 - $10,16% - 20%,4 Stars,Positive,"[I, chose, Food, Delivery, which, is, a, cheap...",I chose Food Delivery which is a cheap and fas...,"[I, chose, Food, Delivery, which, is, a, cheap...","[i, chose, food, delivery, which, is, a, cheap...","[i, chose, food, delivery, which, is, a, cheap..."


In [85]:
#remove stopwords
#https://www.codegrepper.com/code-examples/python/NameError%3A+name+%27stopwords%27+is+not+defined

import re
import nltk
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
ds['stopwords_removed'] = ds['no_punc'].apply(lambda x: [word for word in x if word not in stop_words])
ds.head()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,response,serv_name,serv_rating,order_freq,pop_density,avg_order,avg_tip,serv_rating,exp_sentiment,no_contract,response_str,tokenized_response,lower,no_punc,stopwords_removed
0,Good Always very hot and very tasty,GrubHub,4 Stars,Less than 4 times a month,Suburban,$21 - $25,16% - 20%,4 Stars,Very positive,"[Good, Always, very, hot, and, very, tasty]",Good Always very hot and very tasty,"[Good, Always, very, hot, and, very, tasty]","[good, always, very, hot, and, very, tasty]","[good, always, very, hot, and, very, tasty]","[good, always, hot, tasty]"
1,When drivers pickup food from the restaurant b...,DoorDash,3 Stars,Less than 4 times a month,Suburban,$30+,16% - 20%,3 Stars,Neutral,"[When, drivers, pickup, food, from, the, resta...",When drivers pickup food from the restaurant b...,"[When, drivers, pickup, food, from, the, resta...","[when, drivers, pickup, food, from, the, resta...","[when, drivers, pickup, food, from, the, resta...","[drivers, pickup, food, restaurant, know, area..."
2,ice cream was cold and I really liked it,DoorDash,5 Stars (Highest),3 - 4 times a week,Suburban,$11 - $15,0% - 5%,5 Stars (Highest),Very positive,"[ice, cream, was, cold, and, I, really, liked,...",ice cream was cold and I really liked it,"[ice, cream, was, cold, and, I, really, liked,...","[ice, cream, was, cold, and, i, really, liked,...","[ice, cream, was, cold, and, i, really, liked,...","[ice, cream, cold, really, liked]"
3,I order food through a third party to receive ...,DoorDash,4 Stars,Less than 4 times a month,Suburban,$21 - $25,6% - 10%,4 Stars,Positive,"[I, order, food, through, a, third, party, to,...",I order food through a third party to receive ...,"[I, order, food, through, a, third, party, to,...","[i, order, food, through, a, third, party, to,...","[i, order, food, through, a, third, party, to,...","[order, food, third, party, receive, chickfila..."
4,I chose Food Delivery which is a cheap and fas...,Postmates,4 Stars,5+ times a week,Suburban,$7 - $10,16% - 20%,4 Stars,Positive,"[I, chose, Food, Delivery, which, is, a, cheap...",I chose Food Delivery which is a cheap and fas...,"[I, chose, Food, Delivery, which, is, a, cheap...","[i, chose, food, delivery, which, is, a, cheap...","[i, chose, food, delivery, which, is, a, cheap...","[chose, food, delivery, cheap, fast, company]"


In [86]:
#determine part of speech for each word to lemmatize; lemmatization comes later
import nltk
nltk.download('averaged_perceptron_tagger')
ds['tagged'] = ds['stopwords_removed'].apply(nltk.tag.pos_tag)
ds.head()

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


,response,serv_name,serv_rating,order_freq,pop_density,avg_order,avg_tip,serv_rating,exp_sentiment,no_contract,response_str,tokenized_response,lower,no_punc,stopwords_removed,tagged
0,Good Always very hot and very tasty,GrubHub,4 Stars,Less than 4 times a month,Suburban,$21 - $25,16% - 20%,4 Stars,Very positive,"[Good, Always, very, hot, and, very, tasty]",Good Always very hot and very tasty,"[Good, Always, very, hot, and, very, tasty]","[good, always, very, hot, and, very, tasty]","[good, always, very, hot, and, very, tasty]","[good, always, hot, tasty]","[(good, JJ), (always, RB), (hot, VBP), (tasty,..."
1,When drivers pickup food from the restaurant b...,DoorDash,3 Stars,Less than 4 times a month,Suburban,$30+,16% - 20%,3 Stars,Neutral,"[When, drivers, pickup, food, from, the, resta...",When drivers pickup food from the restaurant b...,"[When, drivers, pickup, food, from, the, resta...","[when, drivers, pickup, food, from, the, resta...","[when, drivers, pickup, food, from, the, resta...","[drivers, pickup, food, restaurant, know, area...","[(drivers, NNS), (pickup, VBP), (food, NN), (r..."
2,ice cream was cold and I really liked it,DoorDash,5 Stars (Highest),3 - 4 times a week,Suburban,$11 - $15,0% - 5%,5 Stars (Highest),Very positive,"[ice, cream, was, cold, and, I, really, liked,...",ice cream was cold and I really liked it,"[ice, cream, was, cold, and, I, really, liked,...","[ice, cream, was, cold, and, i, really, liked,...","[ice, cream, was, cold, and, i, really, liked,...","[ice, cream, cold, really, liked]","[(ice, NN), (cream, NN), (cold, VBD), (really,..."
3,I order food through a third party to receive ...,DoorDash,4 Stars,Less than 4 times a month,Suburban,$21 - $25,6% - 10%,4 Stars,Positive,"[I, order, food, through, a, third, party, to,...",I order food through a third party to receive ...,"[I, order, food, through, a, third, party, to,...","[i, order, food, through, a, third, party, to,...","[i, order, food, through, a, third, party, to,...","[order, food, third, party, receive, chickfila...","[(order, NN), (food, NN), (third, JJ), (party,..."
4,I chose Food Delivery which is a cheap and fas...,Postmates,4 Stars,5+ times a week,Suburban,$7 - $10,16% - 20%,4 Stars,Positive,"[I, chose, Food, Delivery, which, is, a, cheap...",I chose Food Delivery which is a cheap and fas...,"[I, chose, Food, Delivery, which, is, a, cheap...","[i, chose, food, delivery, which, is, a, cheap...","[i, chose, food, delivery, which, is, a, cheap...","[chose, food, delivery, cheap, fast, company]","[(chose, VB), (food, NN), (delivery, NN), (che..."


In [87]:
#lemmatize with adjustments to speech tags

#https://www.programmersought.com/article/40037609941/
from nltk.corpus import wordnet

def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

ds['wordnet_pos'] = ds['tagged'].apply(lambda x: [(word, get_wordnet_pos(pos_tag)) for (word, pos_tag) in x])
ds.head()

,response,serv_name,serv_rating,order_freq,pop_density,avg_order,avg_tip,serv_rating,exp_sentiment,no_contract,response_str,tokenized_response,lower,no_punc,stopwords_removed,tagged,wordnet_pos
0,Good Always very hot and very tasty,GrubHub,4 Stars,Less than 4 times a month,Suburban,$21 - $25,16% - 20%,4 Stars,Very positive,"[Good, Always, very, hot, and, very, tasty]",Good Always very hot and very tasty,"[Good, Always, very, hot, and, very, tasty]","[good, always, very, hot, and, very, tasty]","[good, always, very, hot, and, very, tasty]","[good, always, hot, tasty]","[(good, JJ), (always, RB), (hot, VBP), (tasty,...","[(good, a), (always, r), (hot, v), (tasty, a)]"
1,When drivers pickup food from the restaurant b...,DoorDash,3 Stars,Less than 4 times a month,Suburban,$30+,16% - 20%,3 Stars,Neutral,"[When, drivers, pickup, food, from, the, resta...",When drivers pickup food from the restaurant b...,"[When, drivers, pickup, food, from, the, resta...","[when, drivers, pickup, food, from, the, resta...","[when, drivers, pickup, food, from, the, resta...","[drivers, pickup, food, restaurant, know, area...","[(drivers, NNS), (pickup, VBP), (food, NN), (r...","[(drivers, n), (pickup, v), (food, n), (restau..."
2,ice cream was cold and I really liked it,DoorDash,5 Stars (Highest),3 - 4 times a week,Suburban,$11 - $15,0% - 5%,5 Stars (Highest),Very positive,"[ice, cream, was, cold, and, I, really, liked,...",ice cream was cold and I really liked it,"[ice, cream, was, cold, and, I, really, liked,...","[ice, cream, was, cold, and, i, really, liked,...","[ice, cream, was, cold, and, i, really, liked,...","[ice, cream, cold, really, liked]","[(ice, NN), (cream, NN), (cold, VBD), (really,...","[(ice, n), (cream, n), (cold, v), (really, r),..."
3,I order food through a third party to receive ...,DoorDash,4 Stars,Less than 4 times a month,Suburban,$21 - $25,6% - 10%,4 Stars,Positive,"[I, order, food, through, a, third, party, to,...",I order food through a third party to receive ...,"[I, order, food, through, a, third, party, to,...","[i, order, food, through, a, third, party, to,...","[i, order, food, through, a, third, party, to,...","[order, food, third, party, receive, chickfila...","[(order, NN), (food, NN), (third, JJ), (party,...","[(order, n), (food, n), (third, a), (party, n)..."
4,I chose Food Delivery which is a cheap and fas...,Postmates,4 Stars,5+ times a week,Suburban,$7 - $10,16% - 20%,4 Stars,Positive,"[I, chose, Food, Delivery, which, is, a, cheap...",I chose Food Delivery which is a cheap and fas...,"[I, chose, Food, Delivery, which, is, a, cheap...","[i, chose, food, delivery, which, is, a, cheap...","[i, chose, food, delivery, which, is, a, cheap...","[chose, food, delivery, cheap, fast, company]","[(chose, VB), (food, NN), (delivery, NN), (che...","[(chose, v), (food, n), (delivery, n), (cheap,..."


In [78]:
#lemmatize the words and the tags
wnl = WordNetLemmatizer()
ds['lemmatized'] = ds['wordnet_pos'].apply(lambda x: [wnl.lemmatize(word, tag) for word, tag in x])
ds.head()

,response,serv_name,no_contract,reponse_str,response_str,tokenized_response,lower,no_punc,stopwords_removed,tagged,wordnet_pos,lemmatized
0,Good Always very hot and very tasty,GrubHub,"[Good, Always, very, hot, and, very, tasty]",Good Always very hot and very tasty,Good Always very hot and very tasty,"[Good, Always, very, hot, and, very, tasty]","[good, always, very, hot, and, very, tasty]","[good, always, very, hot, and, very, tasty]","[good, always, hot, tasty]","[(good, JJ), (always, RB), (hot, VBP), (tasty,...","[(good, a), (always, r), (hot, v), (tasty, a)]","[good, always, hot, tasty]"
1,When drivers pickup food from the restaurant b...,DoorDash,"[When, drivers, pickup, food, from, the, resta...",When drivers pickup food from the restaurant b...,When drivers pickup food from the restaurant b...,"[When, drivers, pickup, food, from, the, resta...","[when, drivers, pickup, food, from, the, resta...","[when, drivers, pickup, food, from, the, resta...","[drivers, pickup, food, restaurant, know, area...","[(drivers, NNS), (pickup, VBP), (food, NN), (r...","[(drivers, n), (pickup, v), (food, n), (restau...","[driver, pickup, food, restaurant, know, area,..."
2,ice cream was cold and I really liked it,DoorDash,"[ice, cream, was, cold, and, I, really, liked,...",ice cream was cold and I really liked it,ice cream was cold and I really liked it,"[ice, cream, was, cold, and, I, really, liked,...","[ice, cream, was, cold, and, i, really, liked,...","[ice, cream, was, cold, and, i, really, liked,...","[ice, cream, cold, really, liked]","[(ice, NN), (cream, NN), (cold, VBD), (really,...","[(ice, n), (cream, n), (cold, v), (really, r),...","[ice, cream, cold, really, like]"
3,I order food through a third party to receive ...,DoorDash,"[I, order, food, through, a, third, party, to,...",I order food through a third party to receive ...,I order food through a third party to receive ...,"[I, order, food, through, a, third, party, to,...","[i, order, food, through, a, third, party, to,...","[i, order, food, through, a, third, party, to,...","[order, food, third, party, receive, chickfila...","[(order, NN), (food, NN), (third, JJ), (party,...","[(order, n), (food, n), (third, a), (party, n)...","[order, food, third, party, receive, chickfila..."
4,I chose Food Delivery which is a cheap and fas...,Postmates,"[I, chose, Food, Delivery, which, is, a, cheap...",I chose Food Delivery which is a cheap and fas...,I chose Food Delivery which is a cheap and fas...,"[I, chose, Food, Delivery, which, is, a, cheap...","[i, chose, food, delivery, which, is, a, cheap...","[i, chose, food, delivery, which, is, a, cheap...","[chose, food, delivery, cheap, fast, company]","[(chose, VB), (food, NN), (delivery, NN), (che...","[(chose, v), (food, n), (delivery, n), (cheap,...","[choose, food, delivery, cheap, fast, company]"


In [88]:
ds.to_csv('OFDS_clean2.csv')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')